In [7]:
# -*- coding: utf-8 -*-

from __future__ import print_function
%matplotlib inline

import matplotlib
from IPython import display as dp

import numpy as np
from skimage import io
import os
import time

# Standard libraries
import os

# Related 3rd party libraries
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

# Local libraries
import utils_kalman as ut
from sort import Sort

#import src

OUTPUT_DIR ='../../../../output'
ROOT_DIR = '../../../../'

# Some constant for the script
TASK = 'task_kalman'
WEEK = 'week3'

# Set useful directories
frames_dir = os.path.join(
    ROOT_DIR,
    'data',
    'm6_week1_frames',
    'frames')

results_dir = os.path.join(OUTPUT_DIR, WEEK, TASK)

# Create folders if they don't exist
if not os.path.isdir(results_dir):
    os.mkdir(results_dir)

# Select input. Options: 'gt_txt', 'gt_xml', 'cnn_out', 'yolo3'

INPUT = 'yolo3'    

# Ground truth file path:

if INPUT == 'gt_txt':
    
    gt_file = os.path.join(ROOT_DIR,
                           'data', 'AICity_data', 'train', 'S03',
                           'c010', 'gt', 'gt.txt')
    
    # Get BBox detection from list    
    df = ut.get_bboxes_from_MOTChallenge(gt_file)
    
    
elif INPUT == 'yolo3':
    
    gt_file = os.path.join(ROOT_DIR,
                           'data', 'AICity_data', 'train', 'S03',
                           'c010', 'det', 'det_yolo3.txt')
    
    # Get BBox detection from list    
    df = ut.get_bboxes_from_MOTChallenge(gt_file)

    
elif INPUT == 'gt_xml':
    
    gt_file = os.path.join(ROOT_DIR,
                           'data', 'AICity_data', 'train', 'S03',
                           'c010', 'gt', 'm6-full_annotation.xml')

    # Get BBox detection from list
    df = ut.get_bboxes_from_aicity_file(gt_file)

    # Adapt GT to final metric calculation (add bbox and track_id columns):
    df.loc[:,'track_id'] = df['id'].values.tolist()

    boxes = []
    for index, row in df.iterrows():
        #boxes.append([row['ytl'], row['xtl'], row['ybr'], row['xbr']])
        boxes.append([row['xtl'], row['ytl'], row['xbr'], row['ybr']])

    df['boxes'] = boxes
    
elif INPUT == 'cnn_out':
    
    gt_file = os.path.join(ROOT_DIR,
                           'data', 'AICity_data', 'train', 'S03',
                           'c010', 'det', 'results_cnn.csv')

    # Get BBox detection from list
    df = pd.read_csv(gt_file)

    # Adapt GT to final metric calculation (add bbox and track_id columns):
    
    boxes = []
    for index, row in df.iterrows():
        #boxes.append([row['ytl'], row['xtl'], row['ybr'], row['xbr']])
        boxes.append([row['xtl'], row['ytl'], row['xbr'], row['ybr']])

    df['boxes'] = boxes


# Display data:
colours = np.random.rand(32,3) # Used only for display

# Sort and group bbox by frame:
df.sort_values(by=['frame'])
df_grouped = df.groupby('frame')

# Run tracker:
display = False
save = False
total_time = 0.0
total_frames = 0
out = []

if display:
    plt.ion()  # for iterative display
    fig, ax = plt.subplots(1, 2, figsize=(20, 20))

# Create instance of the SORT tracker
mot_tracker = Sort()  

for f, df_group in df_grouped:
    
    frame = int(df_group['frame'].values[0])
    
    if INPUT == 'gt_txt' or INPUT == 'yolo3':
        df_gt = df_group[['ymin', 'xmin', 'ymax', 'xmax']].values.tolist()     
        
    elif INPUT == 'gt_xml':
        df_gt = df_group[['ytl', 'xtl', 'ybr', 'xbr']].values.tolist()
        
    elif INPUT == 'cnn_out':
        #ToDo: When we correct the order of CNN output CHANGE this order to the same as gt_xml!!
        df_gt = df_group[['ybr', 'xtl', 'ytl', 'xbr']].values.tolist()   

    # Reshape GT format for Kalman tracking algorithm:
    # [x1,y1,x2,y2] format for the tracker input:
    
    df_gt = np.asarray(df_gt)
    dets = np.stack([df_gt[:,1], df_gt[:,0], df_gt[:,3], df_gt[:,2]], axis=1)
    dets = np.reshape(dets, (len(dets), -1))
    dets = np.asarray(dets, dtype=np.float64, order='C')

    
    if (display):
        fn = '../../../../frames/frame_%03d.jpg' % (frame)  # read the frame
        im = io.imread(fn)
        ax[0].imshow(im)
        ax[0].axis('off')
        ax[0].set_title('Original R-CNN detections (untracked)')
        for j in range(np.shape(dets)[0]):
            color = 'r'
            coords = (dets[j, 0].astype(np.float), dets[j, 1].astype(np.float)), dets[j, 2].astype(np.float) - dets[j, 0].astype(np.float), dets[j, 3].astype(np.float) - dets[j, 1].astype(np.float)
            ax[0].add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, lw=3))

    total_frames += 1
    
    if (display):
        ax[1].imshow(im)
        ax[1].axis('off')
        ax[1].set_title('Tracked Targets')

    start_time = time.time()
    trackers = mot_tracker.update(dets)
    cycle_time = time.time() - start_time
    total_time += cycle_time

    out.append([frame, trackers])

    for d in trackers:
        if (display):
            d = d.astype(np.uint32)
            ax[1].add_patch(
                patches.Rectangle((d[0], d[1]), d[2] - d[0], d[3] - d[1], fill=False, lw=3, ec=colours[d[4] % 32, :]))
            ax[1].set_adjustable('box-forced')
    
    if (save):
        plt.savefig(os.path.join(results_dir, 'video_kalman_' + str(frame) + '.png'))

    if (display):
        dp.clear_output(wait=True)
        dp.display(plt.gcf())
        time.sleep(0.000001)
        ax[0].cla()
        ax[1].cla()

plt.show()

print("Total Tracking took: %.3f for %d frames or %.1f FPS" % (total_time, total_frames, total_frames / total_time))


Total Tracking took: 6.983 for 2141 frames or 306.6 FPS


Transform output to pandas format:

In [8]:
import map_metrics as mp

# Result of Kalman tracking (pandas format):

df_kalman = ut.kalman_out_to_pandas_for_map(out)

if INPUT == 'gt_txt' or INPUT == 'yolo3':
    df_gt = ut.get_bboxes_from_MOTChallenge_for_map(gt_file)
    
elif INPUT == 'gt_xml':
    df_gt = df
    
elif INPUT == 'cnn_out':
    df_gt = None
    
# If ground truth was used, save ground truth adapted to map_metric.py:

if df_gt is not None:
    
    ut.save_pkl(df_gt, os.path.join(results_dir, INPUT + '_gt_panda.pkl'))
    df_gt_corr = ut.panda_to_json_gt(df_gt)
    ut.save_json(df_gt_corr, os.path.join(results_dir, INPUT + '_gt_ground_truth_boxes.json'))
    
# Save kalman filter output:

ut.save_pkl(df_kalman, os.path.join(results_dir, INPUT + '_kalman_predictions.pkl'))
df_pred_corr = ut.panda_to_json_predicted(df_kalman) 
ut.save_json(df_pred_corr, os.path.join(results_dir, INPUT + '_predicted_boxes.json'))



In [ ]:
import pandas as pd
import json


def locate_panda_feature(pd_dataframe, key, value):
    """
    Group pandas dataframe by selected key
    """
    grouped_pd = pd_dataframe.groupby(key).agg(lambda x: list(x))
    
    return grouped_pd.loc[value]


def kalman_out_to_pandas(out_kalman):
    
    """
    :param out_kalman: Output from kalman tracking
    :returns: Panda dataframe with format 'frame', 'ymin', 'xmin', 'ymax', 'xmax', 'track_id'
    """

    vals = list()

    for frame_data in out_kalman:

        frame = frame_data[0]
        frame_vals = [frame]

        for track in frame_data[1]:

            ymin, xmin, ymax, xmax, track_id = track
            
            #score = 1
            score = np.random.uniform(0,1)
            
            frame_vals = [frame, ymin, xmin, ymax, xmax, track_id, score]

            vals.append(frame_vals)

    df_kalman = pd.DataFrame(vals, columns=['frame', 'ymin', 'xmin', 'ymax', 'xmax', 'track_id', 'score'])
    
    return df_kalman

df_kalman = kalman_out_to_pandas(out)



def kalman_out_to_pandas_for_map(out_kalman):
    
    """
    Prepair dictionary for map_metrics.py
    :param out_kalman: Output from kalman tracking
    :returns: Panda dataframe with format 'img_id', 'boxes', 'track_id', 'scores'
    """

    vals = list()

    for frame_data in out_kalman:

        img_id = frame_data[0]
        frame_vals = [frame]

        for track in frame_data[1]:
                       
            xmin, ymin, xmax, ymax, track_id = track
                       
            boxes = [xmin, ymin, xmax, ymax]
            
            scores = np.random.uniform(low=0.8, high=1.0)

            frame_vals = [img_id, boxes, track_id, scores]

            vals.append(frame_vals)

    df_kalman = pd.DataFrame(vals, columns=['img_id', 'boxes', 'track_id', 'scores'])
    
    return df_kalman




def get_bboxes_from_MOTChallenge_for_map(fname):
    """
    Read GT as format required in map_metrics.py
    
    Get the Bboxes from the txt files
    MOTChallengr format [frame,ID,left,top,width,height,1,-1,-1,-1]
     {'ymax': 84.0, 'frame': 90, 'track_id': 2, 'xmax': 672.0, 'xmin': 578.0, 'ymin': 43.0, 'occlusion': 1}
    fname: is the path to the txt file
    :returns: Pandas DataFrame with the data
    """
    f = open(fname,"r")
    BBox_list = list()

    for line in f:
        data = line.split(',')
        xmax = float(data[2])+float(data[4])
        ymax = float(data[3])+float(data[5])
        
        BBox_list.append({'img_id':int(data[0]),
                          'track_id':int(data[1]),
                          'boxes': [float(data[2]), float(data[3]), xmax, ymax],
                          'occlusion': 1,
                          'conf' :float(data[6])})
        
    return pd.DataFrame(BBox_list)




def get_bboxes_from_xml_for_map(gt_panda):

    """
    Prepair dictionary for map_metrics.py
    :param out_kalman: Output from kalman tracking
    :returns: Panda dataframe with format 'img_id', 'boxes', 'track_id'
    """

    vals = list()

    for frame_data in gt_panda:

        img_id = frame_data[0]
        frame_vals = [frame]
        
        boxes = []
        
        for track in frame_data[1]:
            
            ymin, xmin, ymax, xmax, track_id = track
            
            boxes.append([xmin, ymin, xmax, ymax])
            
            scores = np.random.uniform(low=0.8, high=1.0)

        frame_vals = [img_id, boxes, track_id, scores]

        vals.append(frame_vals)

    df_kalman = pd.DataFrame(vals, columns=['img_id', 'boxes', 'track_id', 'scores'])
    
    return df_kalman


####################################################


def panda_to_json_predicted(predicted_panda):
    """
    Transform predicted boxes panda to json file used in map_metrics.py
    """
    d_pred = {}
    for name , group in predicted_panda.groupby('img_id'):
        boxes = []
        scores = []
        for idx in range(len(group)):
            boxes.append(group.iloc[idx].boxes)

            scores.append(group.iloc[idx].scores)
        d_pred[group.iloc[idx].img_id] = {'boxes': boxes, 'scores':scores}
    
    return d_pred


def panda_to_json_gt(gt_panda):
    """
    Transform gt boxes panda to json file used in map_metrics.py
    """    
    d_gt = {}
    for name , group in gt_panda.groupby('img_id'):
        boxes = []
        for idx in range(len(group)):
            boxes.append(group.iloc[idx].boxes)
        d_gt[group.iloc[idx].img_id] = {'boxes': boxes}
    
    return d_gt


def save_json(panda_file, filename):
    with open(filename, 'w') as fp:
        json.dump(panda_file, fp)
        
        
def save_pkl(panda_file, filename):
    panda_file.to_pickle(filename)

